# Treinamento Nerds Viajantes - LDA

In [88]:
%reload_ext autoreload
%autoreload 2

In [76]:
# Modulos da propria aplicacao
from limpeza import limpeza_posts, limpeza_texto
from repository import nerds_viajantes, wikipedia, mongo_utils
from treinamento import dicionario, treinamento_lda
from treinamento.treinamento_lda import TreinamentoLda

from repository.mongo_utils import get_pages_content_collection
from repository.wikipedia import WikipediaRepo

wikipedia_repo = WikipediaRepo(collection=get_pages_content_collection())

In [69]:
from similarity.similarity import SimilarityCalculator

In [27]:
# Modelos de pacotes
import pandas as pd

## Trabalho com base de treinamento

In [28]:
def preparar_documentos_nerds_viajantes():
    # Carregar posts
    published = nerds_viajantes.read_published()

    # Remove posts desnecessarios
    posts = limpeza_posts.limpar_posts(published)

    # Limpa texto e gera documentos para treinamento
    documentos = posts['content'].apply(limpeza_texto.limpar_texto)

    return documentos

documentos = preparar_documentos_nerds_viajantes()

### Ajuste de modelo para posts do Nerds Viajantes

In [80]:
# Ajusta modelo LDA para os posts do nerds viajantes. O resultado gerado contem o dicionario, o corpus de dados e o modelo gerado
treinamento_lda = TreinamentoLda()
lda_nerds_viajantes = treinamento_lda.ajustar_modelo(documentos)

### Definicao de probabilidades de topicos para posts do Nerds Viajantes

In [85]:
probabilidades_topicos_nerds_viajantes = [treinamento_lda.calcular_probabilidades_documento(dnv, lda_nerds_viajantes) for dnv in documentos]

### Teste com classe

## Trabalho com base da Wikipedia

In [10]:
pages = wikipedia_repo.find_all()
pages_df = pd.DataFrame(pages)
pages_df = pages_df.drop(['_id', 'ns', 'type', 'download', 'categories', 'wikitext', 'text'], axis='columns')

In [15]:
# Limpa texto e gera documentos para treinamento
documentos_wikipedia = pages_df['text_clean'].apply(limpeza_texto.limpar_texto)

0    [geograf, argentin, domíni, estud, conhec, sob...
1    [banc, namuncur, banc, burdwood, plataform, su...
Name: text_clean, dtype: object

### Definicao de topicos para documentos da Wikipedia

- Quantidade de documentos: 2252
- Indice do maior documento: 123
- Tamanho do maior documento: 10018 tokens
- Tempo gasto para definir tópicos de maior documento: 0.02167201042175293 segundos
- Tempo estimado para todos os documentos (com base no tempo do maior): 18.83 segundos
    * Na verdade este valor variou muito, não é fixo, mas a maioria das vezes ficou em torno de 20 segundos

In [86]:
import time

def avaliar_tempos_definicao_topicos_wikipedia():
    print(f'Quantidade de documentos: {len(documentos_wikipedia)}')

    tamanhos = [len(d) for d in documentos_wikipedia]
    indice_maior_documento = np.argmax(tamanhos)
    print(f'Indice do maior documento: {indice_maior_documento}')
    print(f'Tamanho do maior documento: {tamanhos[indice_maior_documento]} tokens')

    maior_documento = documentos_wikipedia[indice_maior_documento]
    start_time = time.time()
    treinamento_lda.calcular_probabilidades_documento(maior_documento, lda_nerds_viajantes)
    tempo_gasto_maior_documento = time.time() - start_time
    print(f"Tempo gasto na definicao dos topicos do maior documento: {tempo_gasto_maior_documento:.2f} segundos")
    print(f'Tempo estimado para todos documentos: {len(documentos_wikipedia) * tempo_gasto_maior_documento:.2f} segundos')

# avaliar_tempos_definicao_topicos_wikipedia()

### Calculo de probabilidade de topicos para paginas da wikipedia

In [87]:
start_time = time.time()
probabilidades_topicos_wikipedia = [
    treinamento_lda.calcular_probabilidades_documento(dw, lda_nerds_viajantes) for dw in documentos_wikipedia]
elapsed_time = time.time() - start_time
print(f'Tempo total gasto na definicao dos topicos: {elapsed_time:.2f} segundos')

Tempo total gasto na definicao dos topicos: 1.46 segundos


## Calculo de semelhanca entre posts do Nerds Viajantes e Posts da Wikipedia

In [68]:
print(f'Quantidade de posts do nerds viajantes: {len(probabilidades_topicos_nerds_viajantes)}')
print(f'Tamanho do array de probabilidades de posts do nerds viajantes: {len(probabilidades_topicos_nerds_viajantes[0])}')
print(f'Quantidade de paginas da wikipedia: {len(probabilidades_topicos_wikipedia)}')
print(f'Tamanho do array de probabilidades de paginas da wikipedia: {len(probabilidades_topicos_wikipedia[0])}')

Quantidade de posts do nerds viajantes: 378
Tamanho do array de probabilidades de posts do nerds viajantes: 13
Quantidade de paginas da wikipedia: 2252
Tamanho do array de probabilidades de paginas da wikipedia: 13


In [89]:
similarity_calculator = SimilarityCalculator(lda_nerds_viajantes)


/home/helder/estudos/tcc-pucmg-2/src/python/notebooks/similarity/similarity.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  doc_topic_dist = np.array([[tup[1] for tup in lst] for lst in self.modelo_lda[self.corpus]])
